In [26]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

load_dotenv()
llm=ChatOpenAI(model="gpt-4o-mini")

llm.invoke([HumanMessage("잘지냈어?")])

AIMessage(content='네, 잘 지냈어요! 당신은 어땠어요? 어떤 이야기든 나눌 수 있어요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 12, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CxmWppB0k1KpxBCjs57s4Qx0EslwK', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bbab4-344e-7ca0-acc7-526e88f924d3-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 12, 'output_tokens': 25, 'total_tokens': 37, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [27]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool(description="타임과 지역명을 받아 현재 시각을 문자열로 반환하는 도구")
#@tool 데코레이터를 사용해 함수를 도구로 등록함
def get_current_time(timezone: str, location: str) -> str:
    tz=pytz.timezone(timezone)
    now=datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time=f'{timezone} ({location}) 현재 시각 {now}'
    #타임존, 지역명, 현재 시각을 문자열로 반환함
    print(location_and_local_time)
    return location_and_local_time


In [28]:
#도구를 tools 리스트에 추가, tool_dict에도 추가
tools=[get_current_time,]
tool_dict={"get_current_time": get_current_time,}

#도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 답변을 생성할 수 있음
llm_with_tools=llm.bind_tools(tools)

In [29]:
from langchain_core.messages import SystemMessage

#사용자의 질문과 도구를 사용해 언어 모델 답변 생성
messages=[
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇 시야?"),
]

#llm_with_tools를 사용해 사용자의 질문에 언어 모델 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

#생성된 언어 모델 답변 출력
print(messages)

#GPT가 함수 한 번만 사용하면 된다고 판단했으므로, response.tool_calls를 확인하면 1개만 들어있음음

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇 시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 84, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CxmWxacqbexhc2Y3JfIySPoaJvZ1v', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bbab4-518d-7693-8a77-f4203ceb51b1-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_akrblFep5Getwtm6CotgTDJk', 'type': 'tool_call'}],

In [30]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    #tool_dict를 사용해 도구 함수 선택
    print(tool_call["args"])
    #도구 호출시 전달된 인자를 출력함
    tool_msg = selected_tool.invoke(tool_call)
    #도구 함수를 호출해 결과를 반환함
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재 시각 2026-01-14 13:12:21


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇 시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 84, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CxmWxacqbexhc2Y3JfIySPoaJvZ1v', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bbab4-518d-7693-8a77-f4203ceb51b1-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_akrblFep5Getwtm6CotgTDJk', 'type': 'tool_call'}

In [31]:
llm_with_tools.invoke(messages)

AIMessage(content='부산의 현재 시각은 2026년 1월 14일 13시 12분 21초입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 140, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CxmX290OvaDIYGbwB3U5cOEsvqKUQ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bbab4-6924-79a0-a962-8b049f277fad-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 140, 'output_tokens': 29, 'total_tokens': 169, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [32]:
#Pydantic: 입력된 데이터의 유효성과 형식을 검증하고, 특정 데이터 형식으로 명확하게 표현할 때 사용하는 라이브러리
#함수의 파라미터 형식을 Pydantic 베이스 모델과 필드를 이용해 더 명확히 정의할 수 있음

from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: AAPL)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: 1d, 1mo, 1y)")

In [33]:
import yfinance as yf

@tool(description="주식 종목의 가격 데이터를 조회하는 함수")
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period = stock_history_input.period)
    history_md = history.to_markdown()

    return history_md

tools = [get_current_time, get_yf_stock_history]
tool_dict = {"get_current_time": get_current_time, "get_yf_stock_history": get_yf_stock_history}

llm_with_tools = llm.bind_tools(tools)

In [34]:
messages.append(HumanMessage("테슬라의 현재 주가와 한 달 전 주가를 비교해줘."))

response = llm_with_tools.invoke(messages)
print(response)
messages.append(response)

content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 230, 'total_tokens': 301, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CxmX9NH1DLRmNWHkrUNbFcuy5NJOG', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019bbab4-83d3-7040-866e-c633df085368-0' tool_calls=[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1d'}}, 'id': 'call_6oCrybsADKKyasKYmNJEPgGm', 'type': 'tool_call'}, {'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_dUhudKltQZQzO5xKgbRqC2ex', 'type': 'tool_call'}] invalid_tool_calls=[] 

In [35]:
#for문 사용하여 gpt가 사용할 함수를 모두 실행하고 메시지에 추가하게 만듦

for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]
    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_msg)

{'stock_history_input': {'ticker': 'TSLA', 'period': '1d'}}
content='| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|\n| 2026-01-13 00:00:00-05:00 |  450.2 | 451.81 | 443.95 |   447.2 | 5.35572e+07 |           0 |              0 |' name='get_yf_stock_history' tool_call_id='call_6oCrybsADKKyasKYmNJEPgGm'
{'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}
content='| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|\n| 2025-12-15 00:00:00-05:00 | 469.44 | 481.77 | 467.66 |  475.31 | 1.14542e+08 |           0 |              0 |\n| 2025-12-16 00:00:00-05:00 | 472.21 | 491.5  | 465.83 |  489.88 | 1.07608e+08 |           0 |              0 

In [36]:
#함수의 실행 결과가 포함된 메시지를 invoke로 다시 처리하여 자연어 형태로 출력

llm_with_tools.invoke(messages)

AIMessage(content='테슬라(TSLA)의 주가는 다음과 같습니다:\n\n- **현재 주가 (2026년 1월 13일)**: $447.20\n- **한 달 전 주가 (2025년 12월 15일)**: $475.31\n\n따라서, 한 달 전 대비 현재 주가는 약 $28.11 하락했습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 1662, 'total_tokens': 1742, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CxmXE5CYtcaqudjG8F47mcCeECUCX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bbab4-95bb-7ee2-bada-82641af96723-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 1662, 'output_tokens': 80, 'total_tokens': 1742, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio':

In [38]:
#stream 방식으로 출력되도록 코드를 추가
for c in llm.stream([HumanMessage("잘 지냈어? 한국 정치치의 문제점이 무엇인지 말해줘.")]):
    print(c.content, end="|")

|한국| 정치|의| 문제|점|은| 여러| 가지|가| 있지만|,| 몇| 가지| 주요|한| 점|들을| 짚|어|볼| 수| 있습니다|.

|1|.| **|정|당| 간| 갈|등|**|:| 정|당| 간|의| 정치|적| 대|립|이| 심|화|되|면서| 정치|적인| 협|상이| 어렵|고|,| 정책|의| 연|속|성이| 떨어|지는| 경|향|이| 있습니다|.| 이는| 정치|적| 불|안|정을| 초|래|하고| 국민|의| 불|신|을| 일|으|킬| 수| 있습니다|.

|2|.| **|정|치|적| 극|단|주의|**|:| 보|수|와| 진|보| 간|의| 극|단|적인| 대|립|이| 심|해|지|면서| 중|립|적|이|거나| 중|도의| 입|장을| 가진| 사람|들의| 목|소|리가| 더욱| 줄|어|들|고| 있습니다|.| 이|로| 인해| 사회|적| 분|열|이| 더욱| 심|화|될| 수| 있습니다|.

|3|.| **|부|패| 문제|**|:| 정치|인|과| 공|직|자|들의| 부|패| 사건|이| 끊|임|없이| 발생|하면서| 국민|의| 정치|에| 대한| 신|뢰|도가| 낮|아|지고| 있습니다|.| 부|패|가| 정치|적| 권|력|의| 남|용|으로| 이어|질| 경우|,| 이는| 민주|주의|의| 근|본|을| 위|협|합니다|.

|4|.| **|중|앙|집|권|적| 결정| 구조|**|:| 정치|적| 결정|이| 중앙|에서| 이루|어|지는| 경|향|이| 있어| 지방|자|치| 및| 지역| 주민|의| 의견|이| 충분|히| 반|영|되지| 않는| 경우|가| 많|습니다|.| 이러한| 구조|는| 지역| 간| 불|균|형|과| 불|만|을| 초|래|할| 수| 있습니다|.

|5|.| **|선|거|제|도의| 문제|**|:| 현재|의| 선|거|제|도|는| 다|당|제를| 지|향|하고| 있지만|,| 여|전히| 일부| 정|당|의| 과|도|한| 의|석| 확보| 문제|나|,| 정|당| 간| 연|합|에| 대한| 비|판|이| 존재|합니다|.| 이는| 소|수|당|이나| 시민|들의| 목|소|리가| 제대로| 반|영|되지| 않|게| 만들| 수| 있습니다|.

|6|.

In [39]:
messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 활용할 수 있어."),
    HumanMessage("부산은 지금 몇 시야?"),
]

response = llm_with_tools.stream(messages)

is_first = True
#첫 실행인지 확인하고, gathered 선언함
# 첫 실행이 아니면 gathered에 내용을 이어붙임

for chunk in response:
    print("chunk type: ", type(chunk))
    if is_first:
        is_first = False
        gathered = chunk
    else:
        gathered += chunk
    
    print("content: ", gathered.content, "tool call chunk", gathered.tool_calls)

#GPT에 요청하기 위해 messages에 추가
messages.append(gathered)
#AIMessageChunk 형식으로 반환됨됨

chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool call chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_gcYrdnqzT57XX4egMe1IUNYU', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool call chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_gcYrdnqzT57XX4egMe1IUNYU', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool call chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_gcYrdnqzT57XX4egMe1IUNYU', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool call chunk [{'name': 'get_current_time', 'args': {'timezone': ''}, 'id': 'call_gcYrdnqzT57XX4egMe1IUNYU', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool call chunk [{'name': 'get_current_time', 'args': {'timezone': 'Asia'}, 'id': 'call_gcYrdnqzT57XX4egMe1IUNYU', 'type': 'tool_c

In [ ]:
gathered
#AIMessageChunk 형식으로 반환됨을 확인할 수 있음

AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model_provider': 'openai', 'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'service_tier': 'default'}, id='lc_run--019bbaba-ee24-7ae2-bd59-eaef8a53fd4c', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': 'Busan'}, 'id': 'call_gcYrdnqzT57XX4egMe1IUNYU', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 153, 'output_tokens': 23, 'total_tokens': 176, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}, tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"Busan"}', 'id': 'call_gcYrdnqzT57XX4egMe1IUNYU', 'index': 0, 'type': 'tool_call_chunk'}], chunk_position='last')

In [41]:
#AIMessageChunk에 과거 대화 내용이 남긴 messages를 추가해 랭체인에서 바로 활용할 수 있음
#함수 실행 과정을 스트림 방식으로 출력할 필요가 없으니 invoke를 사용함

for tool_call in gathered.tool_calls:
    selected_tool = tool_dict[tool_call["name"]]

    print(tool_call["args"])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages
#ToolMessage에 함수 실행 결과가 포함된 것을 확인 가능

{'timezone': 'Asia/Seoul', 'location': 'Busan'}
Asia/Seoul (Busan) 현재 시각 2026-01-14 13:47:23


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 활용할 수 있어.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇 시야?', additional_kwargs={}, response_metadata={}),
 AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model_provider': 'openai', 'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'service_tier': 'default'}, id='lc_run--019bbaba-ee24-7ae2-bd59-eaef8a53fd4c', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': 'Busan'}, 'id': 'call_gcYrdnqzT57XX4egMe1IUNYU', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 153, 'output_tokens': 23, 'total_tokens': 176, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}, tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"Busan"}', 'id': 'call_gcYrdnqzT57XX4egMe1IUNYU', 'index':

In [ ]:
#stream방식으로 출력
for c in llm_with_tools.stream(messages):
    print(c.content, end="|")